## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
#looks like the max temps are in kelvin, we need to convert that column into Farenheit

city_data_df["Max TempF"]=(city_data_df["Max Temp"]-273.15)*(9/5) + 32
city_data_df.head()

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Max TempF
0,0,dikson,73.5069,80.5464,276.67,89,54,5.89,RU,broken clouds,38.336
1,1,balad,34.0164,44.1453,313.07,14,0,4.85,IQ,clear sky,103.856
2,2,vilhena,-12.7406,-60.1458,303.27,39,0,5.66,BR,clear sky,86.216
3,3,busselton,-33.6500,115.3333,287.64,66,44,8.69,AU,scattered clouds,58.082
4,4,rikitea,-23.1203,-134.9692,294.45,65,97,1.75,PF,overcast clouds,70.340


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 80


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preffered_cities=city_data_df.loc[(city_data_df["Max TempF"]>=min_temp) & (city_data_df["Max TempF"]<=max_temp)]
preffered_cities

,City_ID,City,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description,Max TempF
4,4,rikitea,-23.1203,-134.9692,294.45,65,97,1.75,PF,overcast clouds,70.340
5,5,ponta do sol,32.6667,-17.1000,299.66,74,6,2.27,PT,clear sky,79.718
8,8,hasaki,35.7333,140.8333,297.37,79,0,6.36,JP,clear sky,75.596
9,9,atuona,-9.8000,-139.0333,298.36,72,52,10.58,PF,broken clouds,77.378
12,12,kapaa,22.0752,-159.3190,299.33,84,75,0.45,US,light rain,79.124
...,...,...,...,...,...,...,...,...,...,...,...
745,745,abonnema,4.7084,6.7931,299.40,64,95,5.24,NG,overcast clouds,79.250
746,746,aguimes,27.9054,-15.4461,296.50,65,20,13.38,ES,few clouds,74.030
754,754,bereda,43.2686,-7.5406,295.60,81,64,2.68,ES,broken clouds,72.410
762,762,necocli,8.4234,-76.7860,297.21,93,100,3.51,CO,overcast clouds,75.308


In [5]:
# 4a. Determine if there are any empty rows.
# if all the is null counts are "0" then no null cells
preffered_cities.isnull().sum()

City_ID                0
City                   0
Latitude               0
Longitude              0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Country                0
Current Description    0
Max TempF              0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#no rows to drop but if they were to be dropped i would use the following method, see link below 
#https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.DataFrame.dropna.html

#based on the cell below, this DF will be called clean DF
clean_df=preffered_cities.dropna(axis=0,how="any")

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max TempF", "Current Description", "Latitude", "Longitude"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max TempF,Current Description,Latitude,Longitude,Hotel Name
4,rikitea,PF,70.340,overcast clouds,-23.1203,-134.9692,
5,ponta do sol,PT,79.718,clear sky,32.6667,-17.1000,
8,hasaki,JP,75.596,clear sky,35.7333,140.8333,
9,atuona,PF,77.378,broken clouds,-9.8000,-139.0333,
12,kapaa,US,79.124,light rain,22.0752,-159.3190,
15,topana,RO,71.402,scattered clouds,44.8167,24.5167,
20,haileybury,CA,75.776,scattered clouds,47.4490,-79.6373,
28,lata,PT,76.046,broken clouds,40.1629,-8.3327,
31,nikolskoye,RU,71.492,broken clouds,59.7035,30.7861,
34,vila franca do campo,PT,73.814,scattered clouds,37.7167,-25.4333,


In [26]:
# 6a. Set parameters to search for hotels with 5000 meters.
radius= "5000"
type="lodging"

params={"key":g_key,"location":location,"radius":radius,"type":type}
#target_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#hotels=requests.get(target_url,params=params).json()

# 6b. Iterate through the hotel DataFrame.
#use iterrows as below
#https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iterrows.html 
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Latitude"]
    lng = row["Longitude"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    target_url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
  
    # 6e. Make request and retrieve the JSON data from the search.
    hotels=requests.get(target_url,params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print(f"Hotel not found in {lat},{lng}... skipping.")
    

    


Hotel not found in 44.8167,24.5167... skipping.
Hotel not found in -5.3833,39.7... skipping.
Hotel not found in 13.4667,22.2... skipping.
Hotel not found in 51.226000000000006,-58.6502... skipping.
Hotel not found in 46.7281,29.7078... skipping.
Hotel not found in 11.3,-10.7167... skipping.
Hotel not found in 13.2154,18.3353... skipping.
Hotel not found in 42.4308,130.6434... skipping.
Hotel not found in -3.1374,142.3475... skipping.
Hotel not found in 49.2361,135.1742... skipping.
Hotel not found in 44.365,39.3103... skipping.
Hotel not found in 6.5105,18.2733... skipping.
Hotel not found in 9.9022,4.4192... skipping.
Hotel not found in 1.1983,-70.1733... skipping.
Hotel not found in 48.8264,58.1442... skipping.


In [27]:
hotel_df

,City,Country,Max TempF,Current Description,Latitude,Longitude,Hotel Name
4,rikitea,PF,70.340,overcast clouds,-23.1203,-134.9692,People ThankYou
5,ponta do sol,PT,79.718,clear sky,32.6667,-17.1000,Hotel do Campo
8,hasaki,JP,75.596,clear sky,35.7333,140.8333,Hotel Sunrise Choshi
9,atuona,PF,77.378,broken clouds,-9.8000,-139.0333,Villa Enata
12,kapaa,US,79.124,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
...,...,...,...,...,...,...,...
745,abonnema,NG,79.250,overcast clouds,4.7084,6.7931,Corper's Lodge Obonoma
746,aguimes,ES,74.030,few clouds,27.9054,-15.4461,Hotel Villa de Aguimes
754,bereda,ES,72.410,broken clouds,43.2686,-7.5406,Pazo de Verdes
762,necocli,CO,75.308,overcast clouds,8.4234,-76.7860,Hotel Boutique El Turista


In [30]:
# 7. Dropd the rows where there is no Hotel Name.
clean_hotel_df=hotel_df.dropna(axis=0,how="any")

In [32]:
# 8a. Create the output File (CSV)
output_data_file="Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max TempF} °F</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Latitude", "Longitude"]]
locations

,Latitude,Longitude
4,-23.1203,-134.9692
5,32.6667,-17.1000
8,35.7333,140.8333
9,-9.8000,-139.0333
12,22.0752,-159.3190
...,...,...
745,4.7084,6.7931
746,27.9054,-15.4461
754,43.2686,-7.5406
762,8.4234,-76.7860


In [43]:
# 11a. Add a marker layer for each city to the map.

max_temp = hotel_df["Max TempF"]
fig = gmaps.figure(center=(0, 0), zoom_level=1)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=100, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))